In [27]:
import requests
import pandas as pd
import json

def getAmount(base, quote, lpt_amount):
    urlBase='''https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/{base}_{quote}?liquidity={lptAmount}'''.format(base=base,quote=quote,lptAmount=lpt_amount)
    okbusdtr = requests.get(urlBase)
    okbusdt = json.dumps(okbusdtr.json())
    data = json.loads(okbusdt)
    df = pd.DataFrame(data['data'], columns=['denom','amount'])
    df['lqt_amount']=df['amount']
    
    return df

def getPrice(base):
    priceUrl='''https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/{base}_usdt-a2b?liquidity=0.01'''.format(base=base)
    priceR = requests.get(priceUrl)
    prices = json.dumps(priceR.json())
    price_data = json.loads(prices)
    baseAmount=price_data['data'][0]['amount']
    quoteAmount=price_data['data'][1]['amount']
    price=float(quoteAmount)/float(baseAmount)
    df = pd.DataFrame({'denom':base,'price':price},pd.Index(range(1)))
    return df

def getReword(poolname):
    rewordUrl='''https://www.okexcn.com/okexchain/v1/farm/earnings/{poolname}/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj'''.format(poolname=poolname)
    rewordR = requests.get(rewordUrl)
    reword = json.dumps(rewordR.json())
    reword_data = json.loads(reword)
    amount=reword_data['amount_yielded'][0]
    df = pd.DataFrame(amount, columns=['denom','amount'],index = [0])
    df['reword_amount']=df['amount']
    return df


In [87]:

#df1=getAmount("okb-c4d","usdt-a2b",0)
#df2=getAmount("okb-c4d","okt",0)
#df3=getAmount("ethk-c63","okt",0)


rr = requests.get('https://www.okexcn.com/okexchain/v1/accounts/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')
rrData = json.loads(json.dumps(rr.json()))
rrdf = pd.DataFrame(rrData['data']['currencies'])
rrdf.rename(columns={'symbol':'denom','available':'amount'},inplace=True)
rrdf['account_amount']=rrdf['amount']
rrdf

#dfr1=getReword("btck_okt")
#dfr2=getReword("ethk_okt")
#dfr3=getReword("okb_okt")

df = rrdf
df['amount']=df['amount'].astype("float64")
#df['lqt_amount']=df['lqt_amount'].astype("float64")
df['account_amount']=df['account_amount'].astype("float64")
#df['reword_amount']=df['reword_amount'].astype("float64")
dfg=df.groupby('denom').sum()


okb=getPrice("okb-c4d")
btck=getPrice("btck-ba9")
ethk=getPrice("ethk-c63")
okt=getPrice("okt")
usdt = pd.DataFrame({'denom':'usdt-a2b','price':1},pd.Index(range(1)))

price = okb.append(btck).append(ethk).append(okt).append(usdt)
dfprice=pd.merge(dfg, price, on='denom')
dfprice['usdt'] = dfprice['amount']*dfprice['price']

dfprice['usdt'].sum()

KeyError: 'data'

In [84]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
dfprice

,denom,amount,account_amount,lqt_amount,price,usdt
0,btck-ba9,0.47952000,0.47952000,0.00000000,52820.21284362,25328.34846277
1,okb-c4d,0.00000000,0.00000000,0.00000000,13.67327000,0.00000000
2,okt,228.45428862,228.45428862,0.00000000,87.64364485,20022.56653517
3,usdt-a2b,2666.51433781,2666.51433781,0.00000000,1.00000000,2666.51433781


In [67]:
dfprice['usdt'].sum()

189757.06900798483

In [270]:
dfr1

,denom,amount,reword_amount
0,okt,9.919790588199812444,9.919790588199812444


In [109]:
okbusdtr = requests.get('https://www.okexcn.com/okexchain/v1/swap/liquidity/remove_quote/okb-c4d_usdt-a2b?liquidity=0.99880969')

okbusdt = json.dumps(okbusdtr.json())
data = json.loads(okbusdt)
okbAmount=data['data'][0]['amount']
usdtAmount=data['data'][1]['amount']
price=float(usdtAmount)/float(okbAmount)
print ("OKB: ", okbAmount)
print ("USDT: ", usdtAmount)
print ("price: ", price)

OKB:  1489.404022518915920542
USDT:  13670.632226345126764748
price:  9.178592255461366


In [159]:
rr = requests.get('https://www.okexcn.com/okexchain/v1/accounts/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')

rrData = json.loads(json.dumps(rr.json()))
rrdf = pd.DataFrame(rrData['data']['currencies'])
rrdf.rename(columns={'symbol':'denom','available':'amount'},inplace=True)
rrdf

,denom,amount,locked
0,ammswap_okb-c4d_usdt-a2b,0.000000002033978094,0
1,btck-ba9,0.000000005441568435,0
2,ethk-c63,0.000000000000000001,0
3,okb-c4d,0.000000008000152614,0
4,okt,267.075525711718378653,0
5,usdt-a2b,2275.764933267381108469,0
6,yfi-bb5,30000.000000000000000000,0


In [268]:
rrr = requests.get('https://www.okexcn.com/okexchain/v1/farm/earnings/btck_okt/okexchain16y4cauu2uy5vhwcaz8jgsfsm20j67670r9lelj')


In [269]:
rrrData = json.loads(json.dumps(rrr.json()))
rrrData


{'target_block_height': '790754',
 'amount_locked': {'denom': 'ammswap_btck-ba9_okt',
  'amount': '49.610546632517983358'},
 'amount_yielded': [{'denom': 'okt', 'amount': '17.471909215568899234'}]}

In [183]:
amount=rrrData['amount_yielded'][0]
df = pd.DataFrame(amount, columns=['denom','amount'],index = [0])
df

,denom,amount
0,okt,9.919790588199812444


In [49]:

#df1=getAmount("okb-c4d","usdt-a2b",0)
df2=getAmount("okb-c4d","okt",1.65982272)
#df3=getAmount("ethk-c63","okt",0)


rr = requests.get('https://www.okexcn.com/okexchain/v1/accounts/okexchain1z0e52d2raru90uw59pkdm9smp246t0ngn0t9n6')
rrData = json.loads(json.dumps(rr.json()))
rrdf = pd.DataFrame(rrData['data']['currencies'])
rrdf.rename(columns={'symbol':'denom','available':'amount'},inplace=True)
rrdf['account_amount']=rrdf['amount']
rrdf

#dfr1=getReword("btck_okt")
#dfr2=getReword("ethk_okt")
#dfr3=getReword("okb_usdt")

df = df2.append(rrdf)
df['amount']=df['amount'].astype("float64")
df['lqt_amount']=df['lqt_amount'].astype("float64")
df['account_amount']=df['account_amount'].astype("float64")
#df['reword_amount']=df['reword_amount'].astype("float64")
dfg=df.groupby('denom').sum()


okb=getPrice("okb-c4d")
btck=getPrice("btck-ba9")
ethk=getPrice("ethk-c63")
okt=getPrice("okt")
usdt = pd.DataFrame({'denom':'usdt-a2b','price':1},pd.Index(range(1)))

price = okb.append(btck).append(ethk).append(okt).append(usdt)
dfprice=pd.merge(dfg, price, on='denom')
dfprice['usdt'] = dfprice['amount']*dfprice['price']

dfprice['usdt'].sum()

10241.054434626552

In [50]:
dfprice

,denom,amount,lqt_amount,account_amount,price,usdt
0,okb-c4d,372.85192012,372.85192012,0.00000000,12.85891897,4794.47262874
1,okt,65.09126622,57.40042398,7.69084224,83.67607703,5446.58180588


In [15]:
from web3 import Web3

# IPCProvider:
#w3 = Web3(Web3.IPCProvider('./path/to/geth.ipc'))

# HTTPProvider:
#w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/971dfadadf284cc684bb270f59f055de'))
w3 = Web3(Web3.HTTPProvider('http://18.163.193.125:8545'))

# WebsocketProvider:
#w3 = Web3(Web3.WebsocketProvider('wss://mainnet.infura.io/ws/v3/971dfadadf284cc684bb270f59f055de'))
#w3 = Web3(Web3.WebsocketProvider('ws://18.163.193.125:8546'))

In [11]:
w3.isConnected()

True

In [13]:
Contract = w3.eth.contract(abi= '''[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]''')

# later, initialize contracts with the same metadata at different addresses:
contract1 = Contract(address='0x69af81e73A73B40adF4f3d4223Cd9b1ECE623074')

In [18]:
w3.eth.getTransactionReceipt('0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060')

AttributeDict({'blockHash': HexBytes('0x4e3a3754410177e6937ef1f84bba68ea139e8d1a2258c5f85db9f1cd715a1bdd'),
 'blockNumber': 46147,
 'contractAddress': None,
 'cumulativeGasUsed': 21000,
 'from': '0xA1E4380A3B1f749673E270229993eE55F35663b4',
 'gasUsed': 21000,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'root': '0x96a8e009d2b88b1483e6941e6812e32263b05683fac202abc622a3e31aed1957',
 'to': '0x5DF9B87991262F6BA471F09758CDE1c0FC1De734',
 'transactionHash': HexBytes('0x5c504ed432cb51138bcf0

In [19]:
event_topic = Web3.sha3(text="Transfer(address,address,uint256)")
log_filter = {
    'fromBlock': Web3.toHex(4880701),
    'toBlock':   Web3.toHex(4880701),#'latest',
    #'address': '0x744d70fdbe2ba4cf95131626614a1763df805b9e',
    'topics': ["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"]
}
log_filter_id = eth.filter(log_filter).filter_id
eth.getFilterLogs(log_filter_id)


SyntaxError: invalid character in identifier (<ipython-input-19-61aca3029649>, line 3)

In [20]:
contract_address='0x69af81e73A73B40adF4f3d4223Cd9b1ECE623074'
event_filter = w3.eth.filter({"address": contract_address})

In [22]:
event_filter.filter_id

'0xc0ee9be40aa7bb79e826da3f23c80c9b'

In [26]:
log_filter_id = event_filter.filter_id
w3.eth.getFilterLogs(log_filter_id)


[]